In [1]:
#Librerías que usamos para la carga de datos
import json
from pprint import pprint
import os
import os.path
import pandas as pd
import re
import numpy as np
from owlready2 import *

# 1. Preparamos nuestros posts

In [2]:
docs=os.listdir("EntradasTexto/")
path='EntradasTexto/'
entradas=[]

#En el bucle creamos un path porque se encuentra en un carpeta del working directory
for doc in docs:
    path1=os.path.join(path, doc) 
    dt=open(path1, 'r')
    texto = dt.read()
    #print(texto)
    entradas.append(texto)
    dt.close()

entradas[1] #Visualizamos el segundo elemento de la lista

'Another species at the birdbath\nNo Comments »2nd December, 2020; Category: Bird baths, Bush birds, Garden birds, Parrots and Lorikeets\n\nPurple-crowned Lorikeet\nA birdbath delight\nEarlier this week I glanced out of my sunroom window to check whether there were any birds at my birdbaths. I currently have three birdbaths just outside the room, one on the ground, one on a pedestal at about 60cm and one hanging from a tree branch at a height of about 1.5 metres. I was delighted to see a small flock of Purple-crowned Lorikeets having a drink and dipping into the water for a bath. I have just checked my list of species to have visited the birdbaths. This was bird species number 36, in addition to the three reptiles and two mammal species.\n\nAfter a moment or two, I counted up to seven individuals in or near the birdbaths. Soon they were joined by several House Sparrows and three or four New Holland Honeyeaters. There was a sudden screeching and a flurry of wings as a Collared Sparrowha

In [3]:
len(entradas)

10

# 2. Preparamos nuestra lista de pájaros

In [4]:
birdslist = []
links = []

f = open("sparql.json", "r")
content = f.read()
jsondecoded = json.loads(content)
for value in jsondecoded['results']['bindings']:
    links.append(value['x']['value'])
    #print(value['x']['value'])
print('Total enlaces:', len(links)) #Total de enlaces que tenemos

for link in links:
    tmp = link.split("/")
    birdname=tmp[-1]
    birdname = re.sub(r'[-_]', ' ', birdname) #Reemplaza los '-' y '_' por un espacio
    birdname = re.sub(r'[()]', '', birdname) #Elimina los caracteres ()
    birdslist.append(birdname)
    #print(tmp[-1])
    
print('Total nombres de pájaros:', len(birdslist))

Total enlaces: 10000
Total nombres de pájaros: 10000


In [5]:
birdsdf=pd.DataFrame([birdslist, links]).transpose()
birdsdf.columns=['Birds_name','Birds_link']
birdsdf.head()

,Birds_name,Birds_link
0,Actenoides,http://dbpedia.org/resource/Actenoides
1,African goshawk,http://dbpedia.org/resource/African_goshawk
2,African pitta,http://dbpedia.org/resource/African_pitta
3,African red eyed bulbul,http://dbpedia.org/resource/African_red-eyed_b...
4,Alcedo,http://dbpedia.org/resource/Alcedo


# 3. Hacemos el matching

In [6]:
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.blank('en')
matcher = PhraseMatcher(nlp.vocab)

patterns = list(nlp.pipe(birdslist))
matcher.add("Birds", None, *patterns)

allmatches=pd.DataFrame(columns=['Birds'])
matchesDoc=[]
for doc in nlp.pipe(entradas):
    matches = matcher(doc)
    result = [doc[start:end] for match_id, start, end in matches]
    tmp = []
    for word in result:
        tmp.append(word.text)
    x = np.array(tmp)
    tmppd = np.unique(x)
    matchesDoc.append(len(tmppd))
    tmppd = pd.DataFrame(tmppd, columns=['Birds'])
    allmatches=pd.concat([allmatches, tmppd], axis=0, ignore_index=True)
        
print(len(allmatches))
print(allmatches)
print(len(matchesDoc))
print(matchesDoc) #Total Pájaros encontrados por documento

17
         Birds
0       Dunlin
1       Magpie
2       Avocet
3        Booby
4        Eagle
5     Flamingo
6         Gull
7      Hoatzin
8      Pelican
9         Tern
10     Vulture
11       Eagle
12      Hoopoe
13     Redwing
14  Shearwater
15        Gull
16        Gull
10
[1, 0, 1, 0, 9, 0, 0, 4, 1, 1]


In [7]:
#Creamos una columna siguiendo la secuencia de pájaros encontrados por documento
docs = [1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 8, 8, 8, 8, 9, 10]
docs = pd.DataFrame(docs, columns=['Documento'])
print(len(docs))

17


In [8]:
BirdsOfPosts=pd.concat([docs, allmatches], axis=1, ignore_index=True)
BirdsOfPosts.columns=['Post', 'Birds']
BirdsOfPosts

,Post,Birds
0,1,Dunlin
1,3,Magpie
2,5,Avocet
3,5,Booby
4,5,Eagle
5,5,Flamingo
6,5,Gull
7,5,Hoatzin
8,5,Pelican
9,5,Tern


In [9]:
BirdsOfPosts= BirdsOfPosts.set_index('Birds').join(birdsdf.set_index('Birds_name'))
BirdsOfPosts = BirdsOfPosts.sort_values('Post')
print(BirdsOfPosts)

            Post                              Birds_link
Dunlin         1      http://dbpedia.org/resource/Dunlin
Magpie         3      http://dbpedia.org/resource/Magpie
Avocet         5      http://dbpedia.org/resource/Avocet
Pelican        5     http://dbpedia.org/resource/Pelican
Hoatzin        5     http://dbpedia.org/resource/Hoatzin
Tern           5        http://dbpedia.org/resource/Tern
Vulture        5     http://dbpedia.org/resource/Vulture
Flamingo       5    http://dbpedia.org/resource/Flamingo
Eagle          5       http://dbpedia.org/resource/Eagle
Booby          5       http://dbpedia.org/resource/Booby
Gull           5        http://dbpedia.org/resource/Gull
Eagle          8       http://dbpedia.org/resource/Eagle
Hoopoe         8      http://dbpedia.org/resource/Hoopoe
Redwing        8     http://dbpedia.org/resource/Redwing
Shearwater     8  http://dbpedia.org/resource/Shearwater
Gull           9        http://dbpedia.org/resource/Gull
Gull          10        http://

# 4. Creamos nuestra Ontología

In [10]:
#Creamos una ontología
onto = get_ontology("http://test.org/onto.owl")

In [11]:
with onto:
    class Bird(Thing):
        pass
    class Post(Thing):
        pass
    class Mention(Post >> Bird):
        pass   

In [12]:
Dunlin = Bird("Dunlin")
Dunlin.iri = 'http://dbpedia.org/resource/Dunlin'
Magpie = Bird("Magpie")    
Magpie.iri = 'http://dbpedia.org/resource/Magpie'
Avocet = Bird("Avocet") 
Avocet.iri = 'http://dbpedia.org/resource/Avocet'
Pelican = Bird("Pelican") 
Pelican.iri = 'http://dbpedia.org/resource/Pelican'
Hoatzin = Bird("Hoatzin")   
Hoatzin.iri = 'http://dbpedia.org/resource/Hoatzin'
Tern = Bird("Tern") 
Tern.iri = 'http://dbpedia.org/resource/Tern'
Vulture = Bird("Vulture") 
Vulture.iri = 'http://dbpedia.org/resource/Vulture'
Flamingo = Bird("Flamingo") 
Flamingo.iri = 'http://dbpedia.org/resource/Flamingo'
Eagle = Bird("Eagle")    
Eagle.iri = 'http://dbpedia.org/resource/Eagle'
Booby = Bird("Booby") 
Booby.iri = 'http://dbpedia.org/resource/Booby'
Gull = Bird("Gull")
Gull.iri = 'http://dbpedia.org/resource/Gull'
Hoopoe = Bird("Hoopoe")   
Hoopoe.iri = 'http://dbpedia.org/resource/Hoopoe'
Redwing = Bird("Redwing")  
Redwing.iri = 'http://dbpedia.org/resource/Redwing'
Shearwater = Bird("Shearwater")
Shearwater.iri = 'http://dbpedia.org/resource/Shearwater'

In [13]:
#creación de individuales
post1 = Post("TRACKING DUNLIN IN THE FRASER ESTUARY")
post2 = Post("Another species at the birdbath")
post3 = Post("A Family of Currawongs")
post4 = Post("Another Lovebird visit")
post5 = Post("Whirlwind Peru!")
post6 = Post("A Ringneck mealtime")
post7 = Post("A busy pair of Galahs")
post8 = Post("Last-Minute Algarve, Portugal! Oct 26-28, 2021")
post9 = Post("\"Black\" Brant in New Haven Harbor")
post10 = Post("A Murder in New Haven...Shocker!")

In [14]:
#Creación de relaciones
post1.Mention = [Dunlin]
post3.Mention = [Magpie]
post5.Mention = [Avocet, Pelican, Hoatzin, Tern, Vulture, Flamingo, Eagle, Booby, Gull]
post8.Mention = [Eagle, Hoopoe, Redwing, Shearwater]
post9.Mention = [Gull]
post10.Mention = [Gull]

In [15]:
#Guardamos nuesta ontología
onto.save(file="myOntology.xml", format = "rdfxml")

In [16]:
!cat /Users/kellymm/Desktop/MASTER\ DS/Mineria\ de\ dades\ i\ text/SegundaParte/TrabajosSegundaParte/TrabajoBirds/myOntology.xml

<?xml version="1.0"?>
<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
         xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
         xmlns:owl="http://www.w3.org/2002/07/owl#"
         xml:base="http://test.org/onto.owl"
         xmlns="http://test.org/onto.owl#">

<owl:Ontology rdf:about="http://test.org/onto.owl"/>

<owl:ObjectProperty rdf:about="#Mention">
  <rdfs:range rdf:resource="#Bird"/>
  <rdfs:domain rdf:resource="#Post"/>
</owl:ObjectProperty>

<owl:Class rdf:about="#Bird">
  <rdfs:subClassOf rdf:resource="http://www.w3.org/2002/07/owl#Thing"/>
</owl:Class>

<owl:Class rdf:about="#Post">
  <rdfs:subClassOf rdf:resource="http://www.w3.org/2002/07/owl#Thing"/>
</owl:Class>

<Bird rdf:about="http://dbpedia.org/resource/Dunlin">
  <rdf:type rdf:resource="http://www.w3.org/2002/07/owl#NamedIndividual"/>
</Bird>

<Bird rdf:about="http://dbpedia.org/resource/Magpie">
  <rdf:type rdf:resource="http://www.w